# Run the Pipeline On a Video

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black

In [2]:
import pandas as pd
import numpy as np
import sys

sys.path.append("../src/")
import chess
from chess import svg

# from video_pull import pull_video
from tqdm import tqdm

import matplotlib.pylab as plt
import cv2
from cairosvg import svg2png
from utils import get_uci, get_fen_df
from viz import plot_frame_results
from fens_to_board import fens_to_board

In [6]:
# df = pd.read_csv("../data/labels/QmmhE0frqcA.csv")

In [7]:
# df.tail()

In [11]:
# def flip_fen(fen):
#     fields = fen.split(" ")
#     fields[0] = fields[0][::-1]
#     flipped_fen = " ".join(fields)
#     return flipped_fen

In [9]:
# df["fen"] = df["fen"].apply(flip_fen)

# Create Label Extractor Object

In [32]:
from label_extract import VideoBoardExtractor

vbe = VideoBoardExtractor(
    "../data/CoffeeChess/QmmhE0frqcA.mp4",
    gt_board_loc=[10, 405, 445, 835],
    predict_fen=True,
    store_gt_boards=True,
    store_masks=True,
)

vbe.load_videocap()
# vbe.process_video(30 * 60 * 1)
vbe.process_video(-1)

	 Loading model '../tensorflow_chessbot/saved_models/frozen_graph.pb'
	 Model restored.


16584it [07:56, 34.78it/s]

Couldn't find chessboard in image


In [33]:
fen_df = get_fen_df(vbe)

In [ ]:
for frame in fen_df["frame"].unique():
    plot_frame_results(vbe, frame)
    plt.show()

# Loop Through FENs and Recreate Game

In [44]:
# fen_df

In [37]:
fen_df = get_fen_df(vbe, 8)

In [38]:
fen_df2, board = fens_to_board(fen_df)

In [45]:
# board.move_stack

In [213]:
last_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"
fen_df["confirmed_move"] = False
board = chess.Board(last_fen)
board.set_castling_fen("KQkq")
move_count = 0
for i, d in fen_df.iterrows():
    fen = d["fen"]
    frame = d["frame"]
    if last_fen == fen:
        continue
    new_board = chess.Board(fen)
    if move_count % 2 == 0:
        who_moved = "w"
        not_moved = "b"
    else:
        who_moved = "b"
        not_moved = "w"

    move = get_uci(board, new_board, who_moved)
    try:
        # if True:
        board.push_san(move)
        last_fen = fen
        move_count += 1
    except Exception as e:
        try:
            move = get_uci(board, new_board, not_moved)
            board.push_san(move)
            last_fen = fen
            move_count += 1
        except:
            print(f"failed move {move_count} with exception {e} frame {frame}")
            print(fen, move)
            pass
            break
    fen_df.loc[i, "confirmed_move"] = True
    fen_df.loc[i, "move_uci"] = move
    # break

# board1 = chess.Board(fen_df["fen"].values[0])
# board2 = chess.Board(fen_df["fen"].values[1])
# move = get_uci(board0, board1, "w")

# move2 = get_uci(board1, board2, "b")

In [230]:
fen_df2.to_csv("../data/labels/{video_id}.csv", index=False)